In [2]:
import sys
sys.path.append('../')

import os
import random
import json
from src.azure_form_recognizer import FormRecognizer
from tqdm import tqdm
import uuid
import pandas as pd

In [8]:
with open('../data/name_uuid_mapping.json') as f:
    name_uuid_mapping=json.load(f)

In [9]:
name_uuid_mapping

{'demo': 'fe5fcec60f5442f69cd1f71bdba5f44f'}

In [14]:
pdfs=[]
pdf_paths=[]

for i in os.listdir('../data/pdf_data/'):
    if (i not in name_uuid_mapping):
        pdfs.append(i)
        pdf_paths.append('../data/pdf_data/'+i)
print(pdfs, "\n", pdf_paths)

['ABSLI-Assured-Savings-Plan-Brochure-Web-Version-V07.pdf'] 
 ['../data/pdf_data/ABSLI-Assured-Savings-Plan-Brochure-Web-Version-V07.pdf']


In [13]:
for pdf_path, pdf in zip(pdf_paths, pdfs):
    if (pdf != pdf_path.split('/')[-1]):
        print('fail')

In [ ]:
text_extractor = FormRecognizer('','')

In [ ]:
res={}
ids=set(name_uuid_mapping.values())

for pdf_path, pdf in tqdm(zip(pdf_paths, pdfs)):
    pages=text_extractor.extract_content_from_pdf(pdf_path)
    content='\n\n'.join(pages)
    name=pdf.split('.')[0]
    if (content==''):
        print(f"{pdf} skipped due to no content!")
        continue

    id = uuid.uuid4().hex
    while(id in ids):
        print('id re-generated')
        id=uuid.uuid4().hex

    name_uuid_mapping[name]=id

    with open(f'../data/scrapped_data/{id}.json', 'w') as f:
        json.dump(
            {
               'name': name,
               'content': content.strip() 
            }, f
        )
    ids.add(id)

In [ ]:
with open('../data/name_uuid_mapping.json','w') as f:
    json.dump(name_uuid_mapping, f)